# HW02 - Mourning Tweets

## Import Data

In [1]:
# Import Libraries
import nltk
import csv
import os
import numpy as np
import pandas as pd
from IPython.display import clear_output

In [2]:
# Import fnmourning dataset
raw_df = pd.read_csv('./data/fnmourning.csv', sep = ',')

# Glimpse to raw dataset
raw_df.head()

,text,tag,lang,emoticon
0,"Hoy uno de mis tíos falleció por COVID-19, no ...",mourning,es,0
1,"Hoy falleció mi abuela. Y eso, sin velorio, si...",mourning,es,0
2,Muere el primer médico en activo por Covid-19 ...,mourning,es,0
3,Historia de una victoria. Historia de los últi...,mourning,es,1
4,He visto dos personas a las que les tomé cariñ...,mourning,es,0


In [3]:
# Replace string tags with binary tags
raw_df.tag = raw_df.tag.replace('mourning',1)
raw_df.tag = raw_df.tag.replace('no mourning',0)

# Split dataset by language
es_df = raw_df[(raw_df.lang == "es")]
en_df = raw_df[(raw_df.lang == "en")]

## Lexicons

In [4]:
# Tweet tokenizer 
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer(preserve_case=False, reduce_len=True, strip_handles=True)

# Tokenize tweets in ES dataset
es_tweets = []
for tweet in es_df.text:
    es_tweets.append(tt.tokenize(tweet))

# Tokenize tweets in EN dataset
en_tweets = []
for tweet in en_df.text:
    en_tweets.append(tt.tokenize(tweet))

In [5]:
# Build dictionaries
from gensim import corpora

# Convert tweets to ES dictionary
es_dict = corpora.Dictionary(es_tweets)

# Convert tweets to ES dictionary
en_dict = corpora.Dictionary(en_tweets)

# Glimpse to dictionaries
print('ES Dictionary:')
print(es_dict)
print('')
print('EN Dictionary:')
print(en_dict)

ES Dictionary:
Dictionary(32009 unique tokens: [',', '-', '.', '19', 'conciencia']...)

EN Dictionary:
Dictionary(27484 unique tokens: [',', '-', '.', '19', 'and']...)


In [6]:
# Tweet corpus

# Count terms in each class


## Classifiers

## Feature Importance